## 1. Configuración inicial

Importamos las librerías necesarias y configuramos el entorno.

In [ ]:
# Librerías estándar de Python científico
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Para hacer el ejemplo reproducible
np.random.seed(42)

print("✓ Librerías cargadas correctamente")

## 2. Generación de datos sintéticos

Para este ejemplo, crearemos una serie temporal artificial con propiedades conocidas:

- **Tendencia**: Crecimiento lineal del 0.5% por observación
- **Estacionalidad diaria**: Ciclo de 24 horas (periodo = 24)
- **Estacionalidad semanal**: Ciclo de 7 días (periodo = 168 horas)
- **Ruido**: Variabilidad aleatoria

In [ ]:
# Parámetros de la simulación
n_hours = 24 * 30  # 30 días de datos horarios
t = np.arange(n_hours)

# Componente de tendencia (crecimiento lineal)
trend = 100 + 0.5 * t

# Estacionalidad diaria (ciclo de 24 horas)
# Simulamos mayor actividad durante el día (8-22h)
daily_seasonal = 15 * np.sin(2 * np.pi * t / 24 - np.pi/2)

# Estacionalidad semanal (ciclo de 7 días)
# Simulamos menor actividad los fines de semana
weekly_seasonal = 10 * np.sin(2 * np.pi * t / (24*7))

# Ruido aleatorio (normal con desviación estándar = 5)
noise = np.random.normal(0, 5, n_hours)

# Serie temporal completa = suma de componentes
y = trend + daily_seasonal + weekly_seasonal + noise

# Crear DataFrame para facilitar el manejo
df = pd.DataFrame({
    'hora': pd.date_range('2025-01-01', periods=n_hours, freq='h'),
    'valor': y,
    'tendencia': trend,
    'estac_diaria': daily_seasonal,
    'estac_semanal': weekly_seasonal,
    'ruido': noise
})

print(f"Serie temporal generada: {n_hours} observaciones")
print(f"Periodo: {df['hora'].min()} a {df['hora'].max()}")
df.head()

## 3. Visualización de la serie temporal

Antes de analizar, siempre es útil visualizar los datos.

In [ ]:
# Gráfico de la serie completa
fig, ax = plt.subplots(figsize=(14, 5))
ax.plot(df['hora'], df['valor'], linewidth=0.8, alpha=0.8)
ax.set_title('Serie Temporal Completa', fontsize=14, fontweight='bold')
ax.set_xlabel('Fecha')
ax.set_ylabel('Valor')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("Observación: La serie muestra fluctuaciones regulares (estacionalidades) sobre una tendencia creciente")

## 4. Análisis exploratorio: Detección de estacionalidades

Usamos la **función de autocorrelación (ACF)** para identificar patrones repetitivos en los datos.

La ACF mide la correlación entre la serie y versiones de sí misma desplazadas en el tiempo (lags).

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf

# Calcular y graficar ACF
fig, ax = plt.subplots(figsize=(14, 5))
plot_acf(df['valor'], lags=200, ax=ax, alpha=0.05)
ax.set_title('Función de Autocorrelación (ACF)', fontsize=14, fontweight='bold')
ax.set_xlabel('Lag (horas)')

# Marcar los períodos esperados
ax.axvline(x=24, color='red', linestyle='--', alpha=0.5, label='Periodo diario (24h)')
ax.axvline(x=168, color='orange', linestyle='--', alpha=0.5, label='Periodo semanal (168h)')
ax.legend()
plt.tight_layout()
plt.show()

print("Interpretación:")
print("- Picos significativos en lag=24, 48, 72... indican estacionalidad diaria")
print("- Patrón de envolvente sugiere estacionalidad semanal")

## 5. Descomposición de la serie

Ahora descomponemos la serie en sus componentes. Usamos un método clásico (STL) como referencia.

In [ ]:
from statsmodels.tsa.seasonal import STL

# Descomposición STL con periodo diario (24 horas)
stl = STL(df['valor'], seasonal=25, period=24)
result = stl.fit()

# Visualización de componentes
fig = result.plot()
fig.set_size_inches(14, 10)
fig.suptitle('Descomposición STL (Periodo = 24 horas)', fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.show()

print("Componentes extraídas:")
print("1. Tendencia: Patrón de largo plazo")
print("2. Estacionalidad: Patrón repetitivo (periodo fijo = 24h)")
print("3. Residuos: Variabilidad no explicada")

## 6. Evaluación de la descomposición

¿Qué tan bien capturó el método las componentes verdaderas?

In [ ]:
# Calcular error de reconstrucción
reconstructed = result.trend + result.seasonal + result.resid
reconstruction_error = np.sqrt(np.mean((df['valor'] - reconstructed)**2))

print(f"Error de reconstrucción (RMSE): {reconstruction_error:.4f}")
print("\nEl valor es muy pequeño, indicando que la reconstrucción es casi perfecta.")

# Comparar tendencia estimada vs real
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# Tendencia
axes[0].plot(df['hora'], df['tendencia'], label='Tendencia real', linewidth=2, alpha=0.7)
axes[0].plot(df['hora'], result.trend, label='Tendencia estimada (STL)', 
             linewidth=2, linestyle='--', alpha=0.7)
axes[0].set_title('Comparación: Tendencia', fontweight='bold')
axes[0].set_ylabel('Valor')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Estacionalidad diaria
axes[1].plot(df['hora'][:168], df['estac_diaria'][:168], label='Estacionalidad real', 
             linewidth=2, alpha=0.7)
axes[1].plot(df['hora'][:168], result.seasonal[:168], label='Estacionalidad estimada (STL)', 
             linewidth=2, linestyle='--', alpha=0.7)
axes[1].set_title('Comparación: Estacionalidad (primeros 7 días)', fontweight='bold')
axes[1].set_xlabel('Fecha')
axes[1].set_ylabel('Valor')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nNota: STL captura bien la tendencia y la estacionalidad diaria principal.")
print("Sin embargo, no captura explícitamente la estacionalidad semanal (está en los residuos).")

## 7. Análisis de residuos

Los residuos deben ser puramente aleatorios (ruido blanco) si la descomposición es buena.

In [ ]:
# Estadísticas de los residuos
residuals = result.resid

fig, axes = plt.subplots(1, 2, figsize=(14, 4))

# Histograma
axes[0].hist(residuals.dropna(), bins=30, edgecolor='black', alpha=0.7)
axes[0].set_title('Distribución de Residuos', fontweight='bold')
axes[0].set_xlabel('Valor del residuo')
axes[0].set_ylabel('Frecuencia')
axes[0].axvline(x=0, color='red', linestyle='--', linewidth=2)

# ACF de residuos
plot_acf(residuals.dropna(), lags=50, ax=axes[1], alpha=0.05)
axes[1].set_title('ACF de Residuos', fontweight='bold')

plt.tight_layout()
plt.show()

print("Interpretación:")
print("- Histograma centrado en 0: ✓ Los residuos tienen media ~0")
print("- ACF sin patrones claros: ✓ Los residuos parecen aleatorios")
print("\nConclusión: La descomposición capturó la mayor parte de la estructura en los datos.")

## 8. Aplicación práctica: Predicción

Una vez descompuesta la serie, podemos extrapolar la tendencia y estacionalidad para hacer predicciones.

In [ ]:
# Extender la tendencia linealmente
n_forecast = 72  # Predecir 3 días adicionales
last_trend_value = result.trend.iloc[-1]
trend_slope = (result.trend.iloc[-1] - result.trend.iloc[-25]) / 24  # Pendiente reciente
forecast_trend = last_trend_value + trend_slope * np.arange(1, n_forecast + 1)

# Repetir patrón estacional
seasonal_pattern = result.seasonal.values[-24:]  # Último ciclo diario
forecast_seasonal = np.tile(seasonal_pattern, n_forecast // 24 + 1)[:n_forecast]

# Predicción = tendencia + estacionalidad
forecast = forecast_trend + forecast_seasonal

# Fechas para la predicción
forecast_dates = pd.date_range(df['hora'].iloc[-1] + pd.Timedelta(hours=1), 
                                periods=n_forecast, freq='h')

# Visualización
fig, ax = plt.subplots(figsize=(14, 5))
ax.plot(df['hora'], df['valor'], label='Datos observados', linewidth=1.5, alpha=0.7)
ax.plot(forecast_dates, forecast, label='Predicción', linewidth=2, 
        linestyle='--', color='red', alpha=0.8)
ax.axvline(x=df['hora'].iloc[-1], color='gray', linestyle=':', linewidth=2, 
           label='Inicio de predicción')
ax.set_title('Serie Temporal con Predicción a 3 días', fontsize=14, fontweight='bold')
ax.set_xlabel('Fecha')
ax.set_ylabel('Valor')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("La predicción extiende la tendencia y repite el patrón estacional.")
print("En un caso real, incluiríamos intervalos de confianza para cuantificar incertidumbre.")

## 9. Conclusiones y próximos pasos

### Lo que aprendimos:

1. **Descomposición de series temporales** permite separar tendencias, estacionalidades y ruido
2. **STL es un método clásico robusto**, pero tiene limitaciones:
   - Solo maneja una estacionalidad a la vez
   - Requiere especificar el periodo de antemano
3. **Visualización es esencial** para entender y validar los resultados

### Limitaciones de este ejemplo:

- Usamos datos sintéticos (propiedades conocidas)
- Solo consideramos estacionalidad diaria explícitamente
- La estacionalidad semanal quedó parcialmente en los residuos

### ¿Qué sigue?

El método de **descomposición adaptativa** que desarrollo en mi tesis:

- ✅ Detecta automáticamente **múltiples estacionalidades** (diaria + semanal + anual)
- ✅ No requiere especificar los periodos de antemano
- ✅ Se adapta a cambios en la magnitud de las componentes

Esto es especialmente útil en series reales complejas como consumo energético, tráfico web, datos climáticos, etc.

---

**Recursos adicionales:**

- Documentación de [statsmodels](https://www.statsmodels.org/stable/tsa.html)
- Libro online: [Forecasting: Principles and Practice](https://otexts.com/fpp3/)
- Mi artículo (en preparación): "Adaptive decomposition for time series with multiple seasonalities"

---

*Este notebook es material complementario a mi investigación doctoral. Para más información, visita [mi portfolio](../index.qmd).*